In [1]:
import pandas as pd
import os
from datetime import datetime

In [6]:
class SymbolsUpdate(object):
    def __init__(self):
        self.database_file = 'database.csv'
        self.country_codes = {
            "US": "US",
            "GB": "United Kingdom",
            "FR": "France",
            "BE": "Belgium",
            "IT": "Italy"
        }

    def load_new_data_from_file(self, file_path: str):
        file_extension = os.path.splitext(file_path)[1].lower()
        
        if file_extension == '.csv':
            new_data = pd.read_csv(file_path)
        elif file_extension in ['.xls', '.xlsx']:
            new_data = pd.read_excel(file_path)
        else:
            print(f"Unsupported file type: {file_extension}. Skipping file.")
            return pd.DataFrame()

        processed_data = []

        for idx, row in new_data.iterrows():
            symbol = row['symbol']
            hold = row['hold']
            country_code = row['isin'][:2]
            country = self.country_codes.get(country_code, 'Unknown')

            cusip_entry = {
                'Symbol': symbol,
                'Hold': hold,
                'Country': country,
                'Item': 'cusip',
                'Item_value': row['cusip'],
                'Updatedby': 'Petroineos',
                'Updatetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            processed_data.append(cusip_entry)

            isin_entry = {
                'Symbol': symbol,
                'Hold': hold,
                'Country': country,
                'Item': 'isin',
                'Item_value': row['isin'],
                'Updatedby': 'Petroineos',
                'Updatetime': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }
            processed_data.append(isin_entry)

        return pd.DataFrame(processed_data)

    def save_new_data(self, input_data: pd.DataFrame):
        if input_data.empty:
            print("No data to save.")
            return

        if os.path.exists(self.database_file):
            database = pd.read_csv(self.database_file)
        else:
            database = pd.DataFrame(columns=input_data.columns)

        symbols_to_update = input_data['Symbol'].unique()
        database = database[~database['Symbol'].isin(symbols_to_update)]

        updated_database = pd.concat([database, input_data], ignore_index=True)
        updated_database.sort_values(by='Symbol', inplace=True)
        updated_database.reset_index(drop=True, inplace=True)

        # Save to CSV
        updated_database.to_csv(self.database_file, index=False)

    def get_data_from_database(self):
        if os.path.exists(self.database_file):
            database = pd.read_csv(self.database_file)
            latest_data = database.sort_values('Updatetime').groupby(['Symbol', 'Item'], as_index=False).last()
            return latest_data
        else:
            return pd.DataFrame(columns=['Symbol', 'Hold', 'Country', 'Item', 'Item_value', 'Updatedby', 'Updatetime'])

In [7]:
su = SymbolsUpdate()
new_data = su.load_new_data_from_file('symbols_update_1.csv')
su.save_new_data(new_data)
new_data = su.load_new_data_from_file('symbols_update_2.csv')
su.save_new_data(new_data)
new_data = su.load_new_data_from_file('symbols_update_3.csv')
su.save_new_data(new_data)

su.get_data_from_database()

,Symbol,Item,Hold,Country,Item_value,Updatedby,Updatetime
0,AAAA,cusip,0,US,A234AC,Petroineos,2024-05-19 12:19:56
1,AAAA,isin,0,US,US01222911,Petroineos,2024-05-19 12:19:56
2,BBBB,cusip,1,United Kingdom,123998,Petroineos,2024-05-19 12:19:56
3,BBBB,isin,1,United Kingdom,GB12222201,Petroineos,2024-05-19 12:19:56
4,CCCC,cusip,1,US,G129111,Petroineos,2024-05-19 12:19:56
5,CCCC,isin,1,US,US01239811,Petroineos,2024-05-19 12:19:56
6,DDDD,cusip,1,Italy,78321,Petroineos,2024-05-19 12:19:56
7,DDDD,isin,1,Italy,IT92812323,Petroineos,2024-05-19 12:19:56
8,GGGG,cusip,1,Belgium,B54334AC,Petroineos,2024-05-19 12:19:56
9,GGGG,isin,1,Belgium,BE012568156,Petroineos,2024-05-19 12:19:56
